<a href="https://colab.research.google.com/github/HwayoungYoon/major-predict-test-duksung/blob/master/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. 환경설정 및 라이브러리 import
<참고>
https://zzsza.github.io/data/2018/08/30/google-colab/

In [ ]:
# Google Drive와 Colab 연동
# 실행 후 나오는 URL 접속하여 authorization code 받기
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd gdrive/My Drive/predict

/content/gdrive/My Drive/predict


In [ ]:
# # 기본
import numpy as np
from numpy.linalg import inv
from numpy.linalg import solve
import random
import pandas as pd
from pandas import Series, DataFrame
import pandas_profiling
import openpyxl

import matplotlib.pyplot as plt
import urllib
import matplotlib.dates as mdates
from matplotlib.dates import bytespdate2num
import matplotlib.ticker as ticker
from matplotlib import font_manager, rc
from matplotlib import style
import seaborn as sns

import pickle

import time

# Machine Learning
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.metrics import (roc_curve, auc, accuracy_score, roc_auc_score)
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.datasets import make_blobs
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from xgboost import XGBClassifier, plot_importance
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import auc, f1_score, accuracy_score
from sklearn.metrics import roc_auc_score

# 설정
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 1. 데이터 준비

## 1) 데이터 입력 및 확인

In [ ]:
# data 입력
df = pd.read_csv("data_merge_plus.csv")

In [ ]:
# data 확인
df.head()

,STUDENT_ID,COLLEGE,MAJOR_NAME,CREDIT_1,CREDIT_2,GPA_1,GPA_2,EXTRA_ACTIVITY,COMPLETE_MAJOR,PF,AR_2016,AR_2017,AR_2018,AR_2019,EVERY_TIME
0,1,Art&Design,동양화,18,12,4.2310,4.2098,300,1,1,15.35,19.85,14.4,10.5,0.111
1,2,Art&Design,동양화,18,18,4.1486,4.1206,50,1,1,15.35,19.85,14.4,10.5,0.111
2,3,Art&Design,동양화,18,15,4.1206,4.1066,300,1,1,15.35,19.85,14.4,10.5,0.111
3,4,Art&Design,동양화,18,18,4.1021,4.0617,300,0,1,15.35,19.85,14.4,10.5,0.111
4,5,Art&Design,동양화,18,18,4.0589,4.0509,200,1,1,15.35,19.85,14.4,10.5,0.111


전공선발 점수 column 'MAJOR_SCORE' 추가

```
MAJOR_SCORE = COMPLETE_MAJOR*400 + EXTRA_ACTIVITY + (CREDIT_1 + CREDIT_2)*150/36
 + (CREDIT_1*GPA_1 + CREDIT_2*GPA_2)/(CREDIT_1 + CREDIT_2)*150/4.5
```

In [ ]:
df['MAJOR_SCORE'] = df['COMPLETE_MAJOR']*400 + df['EXTRA_ACTIVITY'] + (df['CREDIT_1'] + df['CREDIT_2'])*150/36 + (df['CREDIT_1']*df['GPA_1'] + df['CREDIT_2']*df['GPA_2'])/(df['CREDIT_1'] + df['CREDIT_2'])*150/4.5

In [ ]:
# data 확인
df.head()

,STUDENT_ID,COLLEGE,MAJOR_NAME,CREDIT_1,CREDIT_2,GPA_1,GPA_2,EXTRA_ACTIVITY,COMPLETE_MAJOR,PF,AR_2016,AR_2017,AR_2018,AR_2019,EVERY_TIME,MAJOR_SCORE
0,1,Art&Design,동양화,18,12,4.2310,4.2098,300,1,1,15.35,19.85,14.4,10.5,0.111,965.750667
1,2,Art&Design,동양화,18,18,4.1486,4.1206,50,1,1,15.35,19.85,14.4,10.5,0.111,737.820000
2,3,Art&Design,동양화,18,15,4.1206,4.1066,300,1,1,15.35,19.85,14.4,10.5,0.111,974.641212
3,4,Art&Design,동양화,18,18,4.1021,4.0617,300,0,1,15.35,19.85,14.4,10.5,0.111,586.063333
4,5,Art&Design,동양화,18,18,4.0589,4.0509,200,1,1,15.35,19.85,14.4,10.5,0.111,885.163333


In [ ]:
# csv 파일로 저장 및 확인
df.to_csv("data_merge.csv", index=False)
pd.read_csv("data_merge.csv")

,STUDENT_ID,COLLEGE,MAJOR_NAME,CREDIT_1,CREDIT_2,GPA_1,GPA_2,EXTRA_ACTIVITY,COMPLETE_MAJOR,PF,AR_2016,AR_2017,AR_2018,AR_2019,EVERY_TIME,MAJOR_SCORE
0,1,Art&Design,동양화,18,12,4.2310,4.2098,300,1,1,15.35,19.85,14.4,10.5,0.111,965.750667
1,2,Art&Design,동양화,18,18,4.1486,4.1206,50,1,1,15.35,19.85,14.4,10.5,0.111,737.820000
2,3,Art&Design,동양화,18,15,4.1206,4.1066,300,1,1,15.35,19.85,14.4,10.5,0.111,974.641212
3,4,Art&Design,동양화,18,18,4.1021,4.0617,300,0,1,15.35,19.85,14.4,10.5,0.111,586.063333
4,5,Art&Design,동양화,18,18,4.0589,4.0509,200,1,1,15.35,19.85,14.4,10.5,0.111,885.163333


## 2) 데이터 나누기

In [ ]:
# A, B로 data copy
A = df.copy()
B = df.copy()

In [ ]:
# A 확인
A.head()

,STUDENT_ID,COLLEGE,MAJOR_NAME,CREDIT_1,CREDIT_2,GPA_1,GPA_2,EXTRA_ACTIVITY,COMPLETE_MAJOR,PF,AR_2016,AR_2017,AR_2018,AR_2019,EVERY_TIME,MAJOR_SCORE
0,1,Art&Design,동양화,18,12,4.2310,4.2098,300,1,1,15.35,19.85,14.4,10.5,0.111,965.750667
1,2,Art&Design,동양화,18,18,4.1486,4.1206,50,1,1,15.35,19.85,14.4,10.5,0.111,737.820000
2,3,Art&Design,동양화,18,15,4.1206,4.1066,300,1,1,15.35,19.85,14.4,10.5,0.111,974.641212
3,4,Art&Design,동양화,18,18,4.1021,4.0617,300,0,1,15.35,19.85,14.4,10.5,0.111,586.063333
4,5,Art&Design,동양화,18,18,4.0589,4.0509,200,1,1,15.35,19.85,14.4,10.5,0.111,885.163333


In [ ]:
# B 확인
B.head()

,STUDENT_ID,COLLEGE,MAJOR_NAME,CREDIT_1,CREDIT_2,GPA_1,GPA_2,EXTRA_ACTIVITY,COMPLETE_MAJOR,PF,AR_2016,AR_2017,AR_2018,AR_2019,EVERY_TIME,MAJOR_SCORE
0,1,Art&Design,동양화,18,12,4.2310,4.2098,300,1,1,15.35,19.85,14.4,10.5,0.111,965.750667
1,2,Art&Design,동양화,18,18,4.1486,4.1206,50,1,1,15.35,19.85,14.4,10.5,0.111,737.820000
2,3,Art&Design,동양화,18,15,4.1206,4.1066,300,1,1,15.35,19.85,14.4,10.5,0.111,974.641212
3,4,Art&Design,동양화,18,18,4.1021,4.0617,300,0,1,15.35,19.85,14.4,10.5,0.111,586.063333
4,5,Art&Design,동양화,18,18,4.0589,4.0509,200,1,1,15.35,19.85,14.4,10.5,0.111,885.163333


A : 모든 column 포함

B : 사용자가 알지 못하는 정보인 'GPA_2' column 제외한 모든 column 포함

In [ ]:
# B에서 'GPA_2' column 삭제
del B['GPA_2']

In [ ]:
# B에서 column 삭제 확인
B.head()

,STUDENT_ID,COLLEGE,MAJOR_NAME,CREDIT_1,CREDIT_2,GPA_1,EXTRA_ACTIVITY,COMPLETE_MAJOR,PF,AR_2016,AR_2017,AR_2018,AR_2019,EVERY_TIME,MAJOR_SCORE
0,1,Art&Design,동양화,18,12,4.2310,300,1,1,15.35,19.85,14.4,10.5,0.111,965.750667
1,2,Art&Design,동양화,18,18,4.1486,50,1,1,15.35,19.85,14.4,10.5,0.111,737.820000
2,3,Art&Design,동양화,18,15,4.1206,300,1,1,15.35,19.85,14.4,10.5,0.111,974.641212
3,4,Art&Design,동양화,18,18,4.1021,300,0,1,15.35,19.85,14.4,10.5,0.111,586.063333
4,5,Art&Design,동양화,18,18,4.0589,200,1,1,15.35,19.85,14.4,10.5,0.111,885.163333


In [ ]:
# A, B csv 파일로 저장
A.to_csv("dataframe_A.csv", index=False)
B.to_csv("dataframe_B.csv", index=False)

## 3) 데이터 수정

In [ ]:
# 데이터 불러오기
df_a = pd.read_csv("dataframe_A.csv")
df_b = pd.read_csv("dataframe_B.csv")

In [ ]:
# df_a 확인
df_a.head()

,STUDENT_ID,COLLEGE,MAJOR_NAME,CREDIT_1,CREDIT_2,GPA_1,GPA_2,EXTRA_ACTIVITY,COMPLETE_MAJOR,PF,AR_2016,AR_2017,AR_2018,AR_2019,EVERY_TIME,MAJOR_SCORE
0,1,Art&Design,동양화,18,12,4.2310,4.2098,300,1,1,15.35,19.85,14.4,10.5,0.111,965.750667
1,2,Art&Design,동양화,18,18,4.1486,4.1206,50,1,1,15.35,19.85,14.4,10.5,0.111,737.820000
2,3,Art&Design,동양화,18,15,4.1206,4.1066,300,1,1,15.35,19.85,14.4,10.5,0.111,974.641212
3,4,Art&Design,동양화,18,18,4.1021,4.0617,300,0,1,15.35,19.85,14.4,10.5,0.111,586.063333
4,5,Art&Design,동양화,18,18,4.0589,4.0509,200,1,1,15.35,19.85,14.4,10.5,0.111,885.163333


In [ ]:
# df_b 확인
df_b.head()

,STUDENT_ID,COLLEGE,MAJOR_NAME,CREDIT_1,CREDIT_2,GPA_1,EXTRA_ACTIVITY,COMPLETE_MAJOR,PF,AR_2016,AR_2017,AR_2018,AR_2019,EVERY_TIME,MAJOR_SCORE
0,1,Art&Design,동양화,18,12,4.2310,300,1,1,15.35,19.85,14.4,10.5,0.111,965.750667
1,2,Art&Design,동양화,18,18,4.1486,50,1,1,15.35,19.85,14.4,10.5,0.111,737.820000
2,3,Art&Design,동양화,18,15,4.1206,300,1,1,15.35,19.85,14.4,10.5,0.111,974.641212
3,4,Art&Design,동양화,18,18,4.1021,300,0,1,15.35,19.85,14.4,10.5,0.111,586.063333
4,5,Art&Design,동양화,18,18,4.0589,200,1,1,15.35,19.85,14.4,10.5,0.111,885.163333


전공명으로 알 수 있는 column 'COLLEGE', 사용하지 않을 column 'STUDENT_ID', 'MAJOR_NAME' 제거

In [ ]:
# 'STUDENT_ID', 'COLLEGE', 'MAJOR_NAME' column 제거
df_a = df_a.drop(columns=['STUDENT_ID', 'COLLEGE', 'MAJOR_NAME'])
df_b = df_b.drop(columns=['STUDENT_ID', 'COLLEGE', 'MAJOR_NAME'])

In [ ]:
# df_a 확인
df_a.head()

,CREDIT_1,CREDIT_2,GPA_1,GPA_2,EXTRA_ACTIVITY,COMPLETE_MAJOR,PF,AR_2016,AR_2017,AR_2018,AR_2019,EVERY_TIME,MAJOR_SCORE
0,18,12,4.2310,4.2098,300,1,1,15.35,19.85,14.4,10.5,0.111,965.750667
1,18,18,4.1486,4.1206,50,1,1,15.35,19.85,14.4,10.5,0.111,737.820000
2,18,15,4.1206,4.1066,300,1,1,15.35,19.85,14.4,10.5,0.111,974.641212
3,18,18,4.1021,4.0617,300,0,1,15.35,19.85,14.4,10.5,0.111,586.063333
4,18,18,4.0589,4.0509,200,1,1,15.35,19.85,14.4,10.5,0.111,885.163333


In [ ]:
# df_b 확인
df_b.head()

,CREDIT_1,CREDIT_2,GPA_1,EXTRA_ACTIVITY,COMPLETE_MAJOR,PF,AR_2016,AR_2017,AR_2018,AR_2019,EVERY_TIME,MAJOR_SCORE
0,18,12,4.2310,300,1,1,15.35,19.85,14.4,10.5,0.111,965.750667
1,18,18,4.1486,50,1,1,15.35,19.85,14.4,10.5,0.111,737.820000
2,18,15,4.1206,300,1,1,15.35,19.85,14.4,10.5,0.111,974.641212
3,18,18,4.1021,300,0,1,15.35,19.85,14.4,10.5,0.111,586.063333
4,18,18,4.0589,200,1,1,15.35,19.85,14.4,10.5,0.111,885.163333


# 2. 모델링

## 1) Xgboost

### (1) A: 1st Model

In [ ]:
# Pass Fail 여부 확인 위해 Y를 'PF' column으로 지정
X = df_a[list(df_a)[1:]]
y = df_a['PF']

# train, test 설정
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

학습 진행 시작

In [ ]:
# 시간 설정
import time
start = time.time()

# Xgboost Classifier
from xgboost import XGBClassifier, plot_importance
model = XGBClassifier()
model.fit(X_train, y_train)

# 소요된 시간 계산(현재시각 - 시작시간 = 실행시간)
print("실행에 소요된 시간 :", time.time()-start)

실행에 소요된 시간 : 0.08058738708496094


In [ ]:
# model 확인
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
model.predict(X_test)

y_pred = model.predict(X_test)

from sklearn.metrics import f1_score

f1_score(y_test,y_pred, average='weighted')

1.0

#### 1st. GridSearchCV : max_depth, gamma

In [ ]:
my_scorer_f1 = make_scorer(f1_score, greater_is_better = True)
my_scorer_acc = make_scorer(accuracy_score, greater_is_better = True)

In [ ]:
xgb_1 = XGBClassifier(
 learning_rate = 0.1,
 n_estimators = 100,
)

In [ ]:
xgb_params_1 = {
    'max_depth' : [3,5,7],
    'gamma' : [0, 0.1],
} 

In [ ]:
X = df_a[list(df_a)[1:]]
y = df_a['PF']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# 시간 설정
import time
start = time.time()

# GridSearchCV
xgb_grid_1 = GridSearchCV(estimator=xgb_1, param_grid=xgb_params_1, 
                          scoring=my_scorer_f1, cv=4, n_jobs=1, verbose=1)
xgb_grid_1.fit(X_train, y_train)

# 최적의 모델 선정 및 소요된 시간 계산(현재시각 - 시작시간 = 실행시간)
print("Best Score : {}".format(xgb_grid_1.best_score_))
print("Best Params : {}".format(xgb_grid_1.best_params_))
print("실행에 소요된 시간 :", time.time()-start)

Fitting 4 folds for each of 6 candidates, totalling 24 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best Score : 1.0
Best Params : {'gamma': 0, 'max_depth': 3}
실행에 소요된 시간 : 1.357297420501709


[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    1.3s finished


In [ ]:
xgb_grid_1.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
# 표 형태로 추출
results = pd.DataFrame(xgb_grid_1.cv_results_)
results = results.sort_values(by='mean_test_score', ascending=False)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gamma,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.051167,0.001993,0.002671,0.000092,0,3,"{'gamma': 0, 'max_depth': 3}",1.0,1.0,1.0,1.0,1.0,0.0,1
1,0.050213,0.000378,0.002608,0.000058,0,5,"{'gamma': 0, 'max_depth': 5}",1.0,1.0,1.0,1.0,1.0,0.0,1
2,0.050548,0.002114,0.002933,0.000629,0,7,"{'gamma': 0, 'max_depth': 7}",1.0,1.0,1.0,1.0,1.0,0.0,1
3,0.049385,0.000531,0.002624,0.000069,0.1,3,"{'gamma': 0.1, 'max_depth': 3}",1.0,1.0,1.0,1.0,1.0,0.0,1
4,0.048928,0.000130,0.002687,0.000104,0.1,5,"{'gamma': 0.1, 'max_depth': 5}",1.0,1.0,1.0,1.0,1.0,0.0,1


#### 2nd. GridSearchCV : max_depth, subsample

In [ ]:
my_scorer_f1 = make_scorer(f1_score, greater_is_better = True)
my_scorer_acc = make_scorer(accuracy_score, greater_is_better = True)

In [ ]:
xgb_2 = XGBClassifier(
 learning_rate = 0.1,
 n_estimators = 100,
)

In [ ]:
xgb_params_2 = {
    'max_depth' : [3,4,5],
    'subsample' : [0.5, 0.7],
} 

In [ ]:
X = df_a[list(df_a)[1:]]
y = df_a['PF']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# 시간 설정
import time
start = time.time()

# GridSearchCV
xgb_grid_2 = GridSearchCV(estimator=xgb_2, param_grid=xgb_params_2, 
                          scoring=my_scorer_f1, cv=4, n_jobs=1, verbose=1)
xgb_grid_2.fit(X_train, y_train)

# 최적의 모델 선정 및 소요된 시간 계산(현재시각 - 시작시간 = 실행시간)
print("Best Score : {}".format(xgb_grid_1.best_score_))
print("Best Params : {}".format(xgb_grid_1.best_params_))
print("실행에 소요된 시간 :", time.time()-start)

Fitting 4 folds for each of 6 candidates, totalling 24 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best Score : 1.0
Best Params : {'gamma': 0, 'max_depth': 3}
실행에 소요된 시간 : 1.5258922576904297


[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    1.4s finished


In [ ]:
xgb_grid_2.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.5, verbosity=1)

In [ ]:
# 표 형태로 추출
results = pd.DataFrame(xgb_grid_2.cv_results_)
results = results.sort_values(by='mean_test_score', ascending=False)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.058626,0.001787,0.002603,0.000018,3,0.5,"{'max_depth': 3, 'subsample': 0.5}",1.0,1.0,1.0,1.0,1.0,0.0,1
1,0.056181,0.001232,0.002792,0.000342,3,0.7,"{'max_depth': 3, 'subsample': 0.7}",1.0,1.0,1.0,1.0,1.0,0.0,1
2,0.057163,0.000805,0.002598,0.000037,4,0.5,"{'max_depth': 4, 'subsample': 0.5}",1.0,1.0,1.0,1.0,1.0,0.0,1
3,0.056442,0.000476,0.002614,0.000056,4,0.7,"{'max_depth': 4, 'subsample': 0.7}",1.0,1.0,1.0,1.0,1.0,0.0,1
4,0.058221,0.002516,0.002600,0.000038,5,0.5,"{'max_depth': 5, 'subsample': 0.5}",1.0,1.0,1.0,1.0,1.0,0.0,1


#### 3rd. GridSearchCV : learning_rate, n_estimators

In [ ]:
my_scorer_f1 = make_scorer(f1_score, greater_is_better = True)
my_scorer_acc = make_scorer(accuracy_score, greater_is_better = True)

In [ ]:
xgb_model = XGBClassifier(
 max_depth = 4,
 subsample = 0.5,
 gamma = 0
)

In [ ]:
xgb_params_3 = {
    'learning_rate' : [0.01, 0.1, 0.2],
    'n_estimators' : [100, 150, 200]}

In [ ]:
X = df_a[list(df_a)[1:]]
y = df_a['PF']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# 시간 설정
import time
start = time.time()

# Core parameter 설정
xgb_grid_3 = GridSearchCV(xgb_model, param_grid = xgb_params_3,
                          scoring = my_scorer_f1, cv=4, verbose=1)
xgb_grid_3.fit(X_train, y_train)

# 최적의 모델 선정 및 소요된 시간 계산(현재시각 - 시작시간 = 실행시간)
print("Best Score : {}".format(xgb_grid_3.best_score_))
print("Best Params : {}".format(xgb_grid_3.best_params_))
print("실행에 소요된 시간 :", time.time()-start)

Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best Score : 1.0
Best Params : {'learning_rate': 0.01, 'n_estimators': 100}
실행에 소요된 시간 : 3.120469093322754


[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:    3.0s finished


In [ ]:
xgb_grid_3.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.5, verbosity=1)

In [ ]:
# 표 형태로 추출
results = pd.DataFrame(xgb_grid_3.cv_results_)
results = results.sort_values(by='mean_test_score', ascending=False)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.067732,0.001786,0.002682,0.000097,0.01,100,"{'learning_rate': 0.01, 'n_estimators': 100}",1.0,1.0,1.0,1.0,1.0,0.0,1
1,0.096033,0.002542,0.002899,0.000242,0.01,150,"{'learning_rate': 0.01, 'n_estimators': 150}",1.0,1.0,1.0,1.0,1.0,0.0,1
2,0.125283,0.000428,0.002894,0.000040,0.01,200,"{'learning_rate': 0.01, 'n_estimators': 200}",1.0,1.0,1.0,1.0,1.0,0.0,1
3,0.058929,0.001677,0.003119,0.000605,0.1,100,"{'learning_rate': 0.1, 'n_estimators': 100}",1.0,1.0,1.0,1.0,1.0,0.0,1
4,0.075861,0.000134,0.002832,0.000060,0.1,150,"{'learning_rate': 0.1, 'n_estimators': 150}",1.0,1.0,1.0,1.0,1.0,0.0,1


#### 4th. GridSearchCV : n_jobs, gamma, subsample, colsample_bytree n_estimators

In [ ]:
my_scorer_f1 = make_scorer(f1_score, greater_is_better = True)
my_scorer_acc = make_scorer(accuracy_score, greater_is_better = True)

In [ ]:
xgb_model_4th = XGBClassifier(
 n_estimators = 100,
 learning_rate = 0.2
)

In [ ]:
xgb_params_4 = {'n_jobs' : [1, 2, 3, 4, 5],
    'gamma' : [0, 0.01, 0.1],
    'subsample' : [0.5, 0.7],
    'colsample_bytree' : [0.7, 0.9]}

In [ ]:
X = df_a[list(df_a)[1:]]
y = df_a['PF']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# 시간 설정
import time
start = time.time()

# Core parameter 설정
xgb_grid_4 = GridSearchCV(xgb_model_4th, param_grid = xgb_params_4,
                          scoring = my_scorer_f1, cv=4, verbose=1)
xgb_grid_4.fit(X_train, y_train)

# 최적의 모델 선정 및 소요된 시간 계산(현재시각 - 시작시간 = 실행시간)
print("Best Score : {}".format(xgb_grid_4.best_score_))
print("Best Params : {}".format(xgb_grid_4.best_params_))
print("실행에 소요된 시간 :", time.time()-start)

Fitting 4 folds for each of 60 candidates, totalling 240 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best Score : 1.0
Best Params : {'colsample_bytree': 0.7, 'gamma': 0, 'n_jobs': 1, 'subsample': 0.5}
실행에 소요된 시간 : 16.442517042160034


[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed:   16.4s finished


In [ ]:
xgb_grid_4.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.5, verbosity=1)

In [ ]:
# 표 형태로 추출
results = pd.DataFrame(xgb_grid_4.cv_results_)
results = results.sort_values(by='mean_test_score', ascending=False)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_gamma,param_n_jobs,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.051452,0.001791,0.002632,0.000045,0.7,0,1,0.5,"{'colsample_bytree': 0.7, 'gamma': 0, 'n_jobs'...",1.0,1.0,1.0,1.0,1.0,0.0,1
1,0.049397,0.000833,0.002702,0.000109,0.7,0,1,0.7,"{'colsample_bytree': 0.7, 'gamma': 0, 'n_jobs'...",1.0,1.0,1.0,1.0,1.0,0.0,1
32,0.045302,0.000773,0.002949,0.000026,0.9,0,2,0.5,"{'colsample_bytree': 0.9, 'gamma': 0, 'n_jobs'...",1.0,1.0,1.0,1.0,1.0,0.0,1
33,0.045063,0.000193,0.002981,0.000043,0.9,0,2,0.7,"{'colsample_bytree': 0.9, 'gamma': 0, 'n_jobs'...",1.0,1.0,1.0,1.0,1.0,0.0,1
34,0.074098,0.003217,0.002822,0.000106,0.9,0,3,0.5,"{'colsample_bytree': 0.9, 'gamma': 0, 'n_jobs'...",1.0,1.0,1.0,1.0,1.0,0.0,1


#### 5th. GridSearchCV : max_depth, n_estimators

In [ ]:
my_scorer_f1 = make_scorer(f1_score, greater_is_better = True)
my_scorer_acc = make_scorer(accuracy_score, greater_is_better = True)

In [ ]:
xgb_model_5th = XGBClassifier(
 gamma = 0,
 n_jobs = 1,
 subsample = 0.5,
 colsample_bytree = 0.7
)

In [ ]:
xgb_params_5 = {'n_estimators' : [100, 300, 500, 700, 900],
    'max_depth' : [10, 15, 20]}

In [ ]:
X = df_a[list(df_a)[1:]]
y = df_a['PF']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# 시간 설정
import time
start = time.time()

# Core parameter 설정
xgb_grid_5 = GridSearchCV(xgb_model_5th, param_grid = xgb_params_5,
                          scoring = my_scorer_f1, cv=4, verbose=1)
xgb_grid_5.fit(X_train, y_train)

# 최적의 모델 선정 및 소요된 시간 계산(현재시각 - 시작시간 = 실행시간)
print("Best Score : {}".format(xgb_grid_5.best_score_))
print("Best Params : {}".format(xgb_grid_5.best_params_))
print("실행에 소요된 시간 :", time.time()-start)

Fitting 4 folds for each of 15 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best Score : 1.0
Best Params : {'max_depth': 10, 'n_estimators': 100}
실행에 소요된 시간 : 12.069653034210205


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   12.0s finished


In [ ]:
xgb_grid_5.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.5, verbosity=1)

In [ ]:
# 표 형태로 추출
results = pd.DataFrame(xgb_grid_5.cv_results_)
results = results.sort_values(by='mean_test_score', ascending=False)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.066894,0.001595,0.003200,0.000359,10,100,"{'max_depth': 10, 'n_estimators': 100}",1.0,1.0,1.0,1.0,1.0,0.0,1
1,0.127994,0.002847,0.003354,0.000360,10,300,"{'max_depth': 10, 'n_estimators': 300}",1.0,1.0,1.0,1.0,1.0,0.0,1
2,0.198414,0.006014,0.003435,0.000093,10,500,"{'max_depth': 10, 'n_estimators': 500}",1.0,1.0,1.0,1.0,1.0,0.0,1
3,0.261204,0.004774,0.003622,0.000137,10,700,"{'max_depth': 10, 'n_estimators': 700}",1.0,1.0,1.0,1.0,1.0,0.0,1
4,0.326180,0.004110,0.003813,0.000085,10,900,"{'max_depth': 10, 'n_estimators': 900}",1.0,1.0,1.0,1.0,1.0,0.0,1


#### 6th. GridSearchCV : learning rate 내리기

In [ ]:
my_scorer_f1 = make_scorer(f1_score, greater_is_better = True)
my_scorer_acc = make_scorer(accuracy_score, greater_is_better = True)

In [ ]:
xgb_model_6nd = XGBClassifier(
 gamma = 0,
 n_jobs = 1,
 subsample = 0.5,
 colsample_bytree = 0.7,
 n_estimators = 100,
 max_depth = 10
)

In [ ]:
xgb_params_6 = {'learning_rate' : [0.01, 0.05, 0.005, 0.1, 0.2]}

In [ ]:
X = df_a[list(df_a)[1:]]
y = df_a['PF']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# 시간 설정
import time
start = time.time()

# Core parameter 설정
xgb_grid_6 = GridSearchCV(xgb_model_6nd, param_grid = xgb_params_6,
                          scoring = my_scorer_f1, cv=5, verbose=1)
xgb_grid_6.fit(X_train, y_train)

# 최적의 모델 선정 및 소요된 시간 계산(현재시각 - 시작시간 = 실행시간)
print("Best Score : {}".format(xgb_grid_6.best_score_))
print("Best Params : {}".format(xgb_grid_6.best_params_))
print("실행에 소요된 시간 :", time.time()-start)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best Score : 1.0
Best Params : {'learning_rate': 0.01}
실행에 소요된 시간 : 2.1407554149627686


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    2.0s finished


In [ ]:
xgb_grid_6.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.5, verbosity=1)

In [ ]:
# 표 형태로 추출
results = pd.DataFrame(xgb_grid_6.cv_results_)
results = results.sort_values(by='mean_test_score', ascending=False)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.093183,0.002688,0.003066,0.000053,0.01,{'learning_rate': 0.01},1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
1,0.079408,0.000624,0.003075,0.000175,0.05,{'learning_rate': 0.05},1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
2,0.092308,0.002386,0.003031,0.000057,0.005,{'learning_rate': 0.005},1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
3,0.066981,0.001100,0.002866,0.000146,0.1,{'learning_rate': 0.1},1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
4,0.054113,0.001767,0.002700,0.000163,0.2,{'learning_rate': 0.2},1.0,1.0,1.0,1.0,1.0,1.0,0.0,1


#### 7th. GridSearchCV : learning rate 올리기 (2nd)

In [ ]:
my_scorer_f1 = make_scorer(f1_score, greater_is_better = True)
my_scorer_acc = make_scorer(accuracy_score, greater_is_better = True)

In [ ]:
xgb_model_7th = XGBClassifier(
 gamma = 0,
 n_jobs = 1,
 subsample = 0.5,
 colsample_bytree = 0.7,
 n_estimators = 100,
 max_depth = 10
)

In [ ]:
xgb_params_7 = {'learning_rate' : [0.2, 0.25, 0.3, 0.35]}

In [ ]:
X = df_a[list(df_a)[1:]]
y = df_a['PF']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# 시간 설정
import time
start = time.time()

# Core parameter 설정
xgb_grid_7 = GridSearchCV(xgb_model_7th, param_grid = xgb_params_7,
                          scoring = my_scorer_f1, cv=5, verbose=1)
xgb_grid_7.fit(X_train, y_train)

# 최적의 모델 선정 및 소요된 시간 계산(현재시각 - 시작시간 = 실행시간)
print("Best Score : {}".format(xgb_grid_7.best_score_))
print("Best Params : {}".format(xgb_grid_7.best_params_))
print("실행에 소요된 시간 :", time.time()-start)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best Score : 1.0
Best Params : {'learning_rate': 0.2}
실행에 소요된 시간 : 1.139890432357788


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.1s finished


In [ ]:
xgb_grid_7.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.5, verbosity=1)

In [ ]:
# 표 형태로 추출
results = pd.DataFrame(xgb_grid_7.cv_results_)
results = results.sort_values(by='mean_test_score', ascending=False)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.054248,0.003075,0.002514,0.000054,0.2,{'learning_rate': 0.2},1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
1,0.049909,0.001904,0.002473,0.000027,0.25,{'learning_rate': 0.25},1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
2,0.048419,0.001578,0.002487,0.000064,0.3,{'learning_rate': 0.3},1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
3,0.048678,0.001975,0.002510,0.000103,0.35,{'learning_rate': 0.35},1.0,1.0,1.0,1.0,1.0,1.0,0.0,1


### (2) A: 2nd Model

In [ ]:
# Pass Fail 여부 확인 위해 Y를 'PF' column으로 지정
X = df_a[list(df_a)[1:]]
y = df_a['PF']

# train, test 설정
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

학습 진행 시작

In [ ]:
best_xgb_model_A = xgb_grid_7.best_estimator_

In [ ]:
best_xgb_model_A

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.5, verbosity=1)

In [ ]:
start = time.time()   # 시간 설정

best_xgb_model_A.fit(X_train, y_train)

print("실행에 소요된 시간 :", time.time()-start) # 소요된 시간 계산(현재시각 - 시작시간 = 실행시간)

실행에 소요된 시간 : 0.07187962532043457


In [ ]:
best_xgb_model_A.predict(X_test)

y_pred = best_xgb_model_A.predict(X_test)

f1_score(y_test,y_pred, average='weighted')

1.0

In [ ]:
# 모델 저장
pickle.dump(best_xgb_model_A, open("best_XGB_A.sav", 'wb'))

### (3) B: 1st Model

In [ ]:
# Pass Fail 여부 확인 위해 Y를 'PF' column으로 지정
X = df_b[list(df_b)[1:]]
y = df_b['PF']

# train, test 설정
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

학습 진행 시작

In [ ]:
xgb_model_B_1 = xgb_grid_7.best_estimator_

In [ ]:
xgb_model_B_1

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.5, verbosity=1)

In [ ]:
start = time.time()   # 시간 설정

xgb_model_B_1.fit(X_train, y_train)

print("실행에 소요된 시간 :", time.time()-start) # 소요된 시간 계산(현재시각 - 시작시간 = 실행시간)

실행에 소요된 시간 : 0.0732424259185791


In [ ]:
xgb_model_B_1.predict(X_test)

y_pred = xgb_model_B_1.predict(X_test)

f1_score(y_test,y_pred, average='weighted')

1.0

In [ ]:
# 모델 저장
pickle.dump(xgb_model_B_1, open("best_xgb_model_B.sav", 'wb'))

### (4) B: 2nd Model

In [ ]:
# Pass Fail 여부 확인 위해 Y를 'PF' column으로 지정
X = df_b[list(df_b)[1:]]
y = df_b['PF']

# train, test 설정
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

학습 진행 시작

In [ ]:
xgb_model_B_2 = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.02, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.5, verbosity=1)

In [ ]:
xgb_model_B_2

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.02, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.5, verbosity=1)

In [ ]:
start = time.time()   # 시간 설정

xgb_model_B_2.fit(X_train, y_train)

print("실행에 소요된 시간 :", time.time()-start) # 소요된 시간 계산(현재시각 - 시작시간 = 실행시간)

실행에 소요된 시간 : 0.12050724029541016


In [ ]:
xgb_model_B_2.predict(X_test)

y_pred = xgb_model_B_2.predict(X_test)

f1_score(y_test,y_pred, average='weighted')

1.0

In [ ]:
# 모델 저장
pickle.dump(xgb_model_B_2, open("best_XGB_B.sav", 'wb'))

## 2) LightGBM

### (1) A: best_LightGBM_model

In [ ]:
# Pass Fail 여부 확인 위해 Y를 'PF' column으로 지정
X = df_a[list(df_a)[1:]]
y = df_a['PF']

# train, test 설정
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
lgbm_params = {"max_depth": 10, "learning_rate" : 0.2, "num_leaves": 100,  "n_estimators": 100}

In [ ]:
start = time.time()   # 시간 설정

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

model = lgb.train(lgbm_params, lgb_train, 2500, lgb_eval, verbose_eval=10)
print("실행에 소요된 시간 :", time.time()-start) # 소요된 시간 계산(현재시각 - 시작시간 = 실행시간)

실행에 소요된 시간 : 0.05657649040222168


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
roc_auc_score(y_test, y_pred)

1.0

In [ ]:
# 모델 저장
pickle.dump(model, open("best_LightGBM_A.sav", 'wb'))

### (2) B: best_LightGBM_model

In [ ]:
# Pass Fail 여부 확인 위해 Y를 'PF' column으로 지정
X = df_b[list(df_b)[1:]]
y = df_b['PF']

# train, test 설정
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
lgbm_params = {"max_depth": 10, "learning_rate" : 0.2, "num_leaves": 100,  "n_estimators": 100}

In [ ]:
start = time.time()   # 시간 설정

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

model = lgb.train(lgbm_params, lgb_train, 2500, lgb_eval, verbose_eval=10)
print("실행에 소요된 시간 :", time.time()-start) # 소요된 시간 계산(현재시각 - 시작시간 = 실행시간)

실행에 소요된 시간 : 0.06937837600708008


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
roc_auc_score(y_test, y_pred)

1.0

In [ ]:
# 모델 저장
pickle.dump(model, open("best_LightGBM_B.sav", 'wb'))